# Real-time Search for Similar Weather in Different Locations on Earth
The final iteration of this project is a program that will tell the user what city on Earth currently has the most similar weather conditions compared to the user's location. The user will enter the name of their current city, and then the program will look at the weather conditions of thousands of cities across the globe to determine which one is most similar. It will calculate a "similarity index" for each city, with a lower index indicating more similar weather. Then, the program will return the name of the city that has the lowest index to the user, as well as the weather conditions in that city compared to the weather conditions in the user's city.


### 0.
Import required modules

In [ ]:
import requests
import time
from multiprocessing import Pool

### 1.
Get the current temperature, relative humidity, windspeed, and sky condition (clear or not clear) in Minneapolis.\
Convert the sky condition (which is initially a string) into 0 (clear) or 1 (not clear) so that it can be quantitatively compared to the rest of the weather data.

In [ ]:
current_time = time.asctime()   # this is recorded as UTC

mpls = requests.get('https://api.weather.gov/stations/KANE/observations/latest')
mpls = mpls.json()
mplsTemp = mpls['properties']['temperature']['value']
mplsHumidity = mpls['properties']['relativeHumidity']['value']
mplsWindspeed = mpls['properties']['windSpeed']['value']
mplsSkyCondition = mpls['properties']['textDescription']
if 'Clear' in mplsSkyCondition:
  mplsSkyConditionIndex = 0
else:
  mplsSkyConditionIndex = 1

### 2.
Get a list of NOAA zones in the U.S.\
Iterate through the zones and grab the Observation Stations (from which we can grab current weather data).\
Remove duplicates from the list of Observation Stations.

In [ ]:
zones = requests.get('https://api.weather.gov/zones?type=land')
zones = zones.json()

stations = []
for i in range(len(zones['features'])):
    if zones['features'][i]['properties']['state'] != 'MN':         # exclude MN weather stations from comparison
        zoneID = zones['features'][i]['properties']['id']
        observationStations = zones['features'][i]['properties']['observationStations']
        if observationStations != []:
            station = observationStations[0][33:]
            stations.append(station)

stations = list(set(stations))

### 3.
Function to request weather data from a NOAA Observation Station. If it returns a valid response (200), grab the current temperature, relative humidity, windspeed, and sky condition (clear or not clear).\
Convert the sky condition (which is initially a string) into 0 (clear) or 1 (not clear) so that it can be compared quantitatively to the other weather data.\
Store all these values in a list [`station`, `temp`, `relativeHumidity`, `windSpeed`, `skyConditionIndex`, `skyCondition`], and then return this list.

In [ ]:
stationWeatherList = []             # the list that will store weather data for each Observation Station
stationsToCompare = stations[:]     # if we want, we can compare only a subset of the total stations

print(f'\nThere are {len(stationsToCompare)} stations in the U.S. to compare.\n')

def getLatestObservation(station):
    weather = requests.get(f'https://api.weather.gov/stations/{station}/observations/latest')
    if weather.status_code == 200:
        weather = weather.json()
        temp = weather['properties']['temperature']['value']
        relativeHumidity = weather['properties']['relativeHumidity']['value']
        windSpeed = weather['properties']['windSpeed']['value']
        skyCondition = weather['properties']['textDescription']
        if 'Clear' in skyCondition:
          skyConditionIndex = 0
        else:
          skyConditionIndex = 1
        weatherList = [station, temp, relativeHumidity, windSpeed, skyConditionIndex, skyCondition]
        validData = True
        for item in weatherList:
          if item is None:
            validData = False
        if validData == True:
          return weatherList


There are 1496 stations in the U.S. to compare.



### 4.
Request weather data from NOAA.\
Use 4 pools to make 4 API calls at a time.

In [ ]:
start_time = time.time()

my_pool = Pool(4)
stationWeatherList = my_pool.map(getLatestObservation, stationsToCompare)
stationWeatherList = [x for x in stationWeatherList if x is not None]   # remove stations that have no data
my_pool.close()

end_time = time.time()

### 5.
Find the Observation Station that has weather most similar to Minneapolis.

In [ ]:
for station in stationWeatherList:

    # calculate absolute difference in temp
    tempDiff = abs(mplsTemp - station[1])

    # calculate absolute difference in humidity - this is given half the weight that temp is given
    humidityDiff = abs(mplsHumidity - station[2]) / 2

    # calculate absolute difference in windspeed
    windDiff = abs(mplsWindspeed - station[3])

    # calculate difference in sky conditions - this is given 5 times the weight because the only values are 0 (clear skies) or 1 (not clear skies)
    skyConditionDiff = abs(mplsSkyConditionIndex - station[4]) * 5

    # calculate the total score for the station
    totalScore = tempDiff + humidityDiff + windDiff + skyConditionDiff
    station.insert(0, totalScore)

# sort the list of weather stations so that the most similar station is at the front of the list
stationWeatherList.sort()

### 6.
Find the name of the location where the Observation Station is located.\
Print out the current weather in Minneapolis.\
Print out the other Observation Station's name and its current weather.

In [ ]:
closestStation = stationWeatherList[0]
closestStationID = closestStation[1]
closestStationTemp = closestStation[2] * 1.8 + 32       # convert Celsius to Fahrenheit
closestStationHumidity = closestStation[3]
closestStationWindspeed = closestStation[4] / 1.609     # convert km/h to mph
closestStationSkyCondition = closestStation[6]

closestStationName = requests.get(f'https://api.weather.gov/stations/{closestStationID}').json()
closestStationName = closestStationName['properties']['name']

mplsTemp = mplsTemp * 1.8 + 32      # convert Celsius to Fahrenheit
mplsWindspeed = mplsWindspeed / 1.609   # convert km/h to mph

print(f'At {current_time}, the weather in Minneapolis is {round(mplsTemp)} degrees Fahrenheit with {round(mplsHumidity)}% humidity and {round(mplsWindspeed)} mph winds. It is {mplsSkyCondition}.\n')
print(f'The location with the most similar current weather is {closestStationName}.\n')
print(f'The current weather at {closestStationName} is {round(closestStationTemp)} degrees Fahrenheit with {round(closestStationHumidity)}% humidity and {round(closestStationWindspeed)} mph winds. It is {closestStationSkyCondition}.\n')


print(f'The total time to compare {len(stationsToCompare)} cities was {end_time - start_time} seconds.\n')

At Wed Nov 20 00:32:05 2024, the weather in Minneapolis is 48 degrees Fahrenheit with 71% humidity and 11 mph winds. It is Cloudy.

The location with the most similar current weather is Chippewa Valley Regional Airport.

The current weather at Chippewa Valley Regional Airport is 49 degrees Fahrenheit with 74% humidity and 11 mph winds. It is Cloudy.

The total time to compare 1496 cities was 53.385841608047485 seconds.



This MVP features many simplifications compared to the final product. First of all, my MVP uses Minneapolis as the starting city and will only compare cities within the U.S. This allows me to only worry about gathering data from one source: the NOAA. Additionally, although I will be comparing four parameters (air temperature, relative humidity, windspeed, and sky condition) when calculating a 'similarity score' for each location, the way in which these parameters are combined to create the 'similarity score' is not scientific. For example, I have simplified the sky condition parameter to be a binary 0 (clear skies) or 1 (not clear skies). This MVP does not distinguish among clouds, rain, snow, or fog; rather, it classifies all those conditions as 'not clear'.